### Allocate Property Codes And Categories

- allocates categories to expenses, property codes to mortgages and property codes to Beals rent payments.
- requires continuous bank data with property codes populated for expense payments (also the Description field with rent periods for self managed properties).

In [21]:
import numpy as np
import pandas as pd

In [22]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [23]:
path='M:/My Documents/Business/Bugisiw Ltd/TaxReturn/TaxReturn_2020-2021/LLP accounts/'

In [24]:
#df6=pd.read_csv(path+'6045.csv', parse_dates=['Date'], dayfirst=True)
#df3=pd.read_csv(path+'3072.csv', parse_dates=['Date'], dayfirst=True)

In [25]:
df6=pd.read_excel(path+'6045.xlsx',sheet_name='6045')
df6=df6[~df6.Account.isnull()]
df6=df6.astype({"Memo": str})
df3=pd.read_excel(path+'3072.xlsx',sheet_name='3072')
df3=df3[~df3.Account.isnull()]
df3=df3.astype({"Memo": str})

In [26]:
df6['Category']=np.nan
df3['Category']=np.nan

In [27]:
startdate = pd.to_datetime('2020-04-06').date()
enddate = pd.to_datetime('2021-04-05').date()
df6=df6[(df6.Date >= startdate)&(df6.Date <= enddate)]
df3=df3[(df3.Date >= startdate)&(df3.Date <= enddate)]

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


#### Manually allocate Property Codes to expense records and rents (including rent periods)

- need to manually allocate property codes to expense records - this is done as part of the Rent Statement process but it is possible to work directly from a bank download.

#### Allocate Categories to Expenses

In [28]:
def allocate_categories(df_bank):
    # 3072
    drawings3072='BT|ONE.*CALL|SHEPHERDS|AVIVA|ID MOBILE|BOOTS|H3G|HARPUR|LEGAL & GEN|TUCKER MS|Bugisiw|NORTH HERTS  REM'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains(drawings3072,case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('SIBERITE.*BGC',case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('Pers OD',case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='OverdraftFee'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('RETURNED',case=False,regex=True)),'Category']='Maintenance'
    
    # References to df_bank.Description requires that property codes and rental periods are completed - this is populated
    # by the Rent Statements
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('M TUCKER              MTUCKER            STO',case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Natwest'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('DMZ',case=False,regex=True))&(df_bank['Property'].notnull()),'Category']='Maintenance'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('BEALS ESTATE AGENT',case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='BealsRentalIncome'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Description.str.contains('Rent for period',case=False,regex=True))&(df_bank['Property'].notnull()),'Category']='RentalIncome'
        
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Description.str.contains('Rent for period',case=False,regex=True))&(df_bank['Property'].notnull()),'Category']='RentalIncome'
    
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('HAG DOORS',case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Intercompany'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('Water rate',case=False,regex=True))&(df_bank['Property'].notnull()),'Category']='RentalIncome'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Description.str.contains('.*Fox.*Sons.*',case=False,regex=True)),'Category']='AgentFees'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('SEQUENCE|GD3',case=False,regex=True)),'Category']='AgentFees'

    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Description.str.contains('deposit|TDS',case=False,regex=True)),'Category']='Deposit'

    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('HMRC',case=False,regex=True)),'Category']='HMRC'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('PCC COLLECTION|Council\s*Tax',case=False,regex=True)),'Category']='CouncilTax'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('deposit|TDS',case=False,regex=True)),'Category']='Deposit'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('EE LIMITED',case=False,regex=True)),'Category']='MobilePhone'
    #df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('GOECO|COMPANIES HOUSE',case=False,regex=True)),'Category']='LegalAndProfessional'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('BOSWELL|PI PROPERTY',case=False,regex=True)),'Category']='Insurance'
    #df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('POST OFFICE',case=False,regex=True)),'Category']='Postage'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('TAXCALC',case=False,regex=True)),'Category']='Accounting'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory.str.contains('PAYMENT|SPLIT|BCC|REPEATPMT',case=False,regex=True))&(df_bank['Property'].notnull()),'Category']='Maintenance'
    
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains('PAYPAL|childrens|ROCCO|SOFIA',case=False,regex=True)),'Category']='Drawings'

    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='REPEATPMT')&(df_bank.Memo.str.contains('ALHAMBRA RD|ALHAMBRA ROAD|23 HAMPSHIRE|HT SC|FRATTON SC',case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='ServiceCharge'

    intercompany1='13438023|13676978|13686035|23534332|83672832|ALHAMBRA'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='FT')&(df_bank.Memo.str.contains(intercompany1,case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Intercompany'
    intercompany2='KEVIN BARRETT|REPLACEMENT KEYS'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='PAYMENT')&(df_bank.Memo.str.contains(intercompany2,case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Intercompany'

    mortgages='BHAM MIDSHIRES|CHL HOME LOAN|MORTGAGE TRUST|PLATFORM FUNDING|TOPAZ FIN|AMBER HOMELOANS|NRAM|TOPAZ.*SIBERITE|SIBERITE'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Memo.str.contains(mortgages,case=False,regex=True))&(df_bank['Property'].isnull()),'Category']='Mortgage'

    drawings='ARGOS|CREATION|LAND ROVER|PAYPAL'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='DIRECTDEBIT')&(df_bank.Property.isnull())&(df_bank.Memo.str.contains(drawings,case=False,regex=True)),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='PAYMENT')&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='FT')&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='REFUND')&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='REPEATPMT')&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='BCC')&(df_bank['Property'].isnull()),'Category']='Drawings'
    df_bank.loc[(df_bank.Category.isnull())&(df_bank.Subcategory=='CASH')&(df_bank['Property'].isnull()),'Category']='Drawings'



In [29]:
allocate_categories(df6)
allocate_categories(df3)

#### Allocate Property Codes to Mortgages

In [30]:
mortgage_map = {
'^MORTGAGE EXPRESS\s*001872470.*$|^TOPAZ FIN ROSINCA.*131992707.*$':'F1321LON',
'^BHAM MIDSHIRES.*20003649652.*$':'F2321LON',
'^BHAM MIDSHIRES.*020010557032.*$':'F2321LON',    
'^.*907371904.*$':'F3321LON',
'^MORTGAGE EXPRESS\s*001703155.*$|^TOPAZ FIN ROSINCA.*131188407.*$':'F4321LON',
'^MORTGAGE EXPRESS\s*001996624.*$|^TOPAZ FIN ROSINCA.*132514207.*$':'F1169FAW',
'^.*907372200.*$':'F2169FAW',
'^BHAM MIDSHIRES.*20000389757.*$':'F3169FAW',
'^BHAM MIDSHIRES.*020010556389.*$':'F3169FAW',
'^MORTGAGE EXPRESS\s*201648504.*$|^TOPAZ FIN ROSINCA.*137376006.*$':'F1171FAW',
'^MORTGAGE EXPRESS\s*201648513.*$|^TOPAZ FIN ROSINCA.*137376104.*$':'F2171FAW',
'^MORTGAGE EXPRESS\s*201648522.*$|^TOPAZ FIN.*191462710.*$':'F3171FAW',
'^MORTGAGE EXPRESS\s*001647053.*$|^TOPAZ FIN ROSINCA.*130936210.*$':'F2163FRA',
'^MORTGAGE EXPRESS\s*001586614.*$|^TOPAZ FIN ROSINCA.*130654303.*$':'F3163FRA',
'^.*20002731371.*$':'F4163FRA',
'^.*KINGSTON SHOP.*$':'SHOP196KIN',
'^BHAM MIDSHIRES.*20007882432.*$':'196AKIN',
'^.*020007900597.*$':'196BKIN',
'^.*20004110676.*$':'23BHAM',
'^.*631516204.*$':'23CHAM',
'^MORTGAGE TRUST\s*002186317.*$':'F58ALH',
'^PLATFORM FUNDING\s*01050228957650.*$':'F68ALH',
'^PLATFORM FUNDING\s*01050228957685.*$':'F78ALH',
'^AMBER HOMELOANS.*480441702.*$':'F88ALH',
'^SKIPTON.*165905969.*$':'F88ALH',
'^MORTGAGE TRUST\s*002186357.*$':'F1746ALH',
'^MORTGAGE TRUST\s*002186356.*$':'F1846ALH',
'^PLATFORM FUNDING\s*01050228979115.*$':'F1946ALH',
'^PLATFORM FUNDING.*01050229961082.*$':'F2046ALH',
'^.*480016104.*$':'F2146ALH',
'^PLATFORM FUNDING\s*01050229961090.*$':'F2246ALH',
'^PLATFORM FUNDING\s*01050228964703.*$':'F2346ALH',
'^MORTGAGE EXPRESS\s*200633231.*$|^TOPAZ FIN.*190812705.*$':'F31214ALH',
'^.*20009309942.*$':'F41214ALH',
'^MORTGAGE EXPRESS\s*200578729.*$|^TOPAZ FIN.*190778206.*$':'F51214ALH',
'^MORTGAGE TRUST\s*002197389.*$':'F71214ALH',
'^MORTGAGE EXPRESS\s*200637264.*$|^TOPAZ FIN ROSINCA.*134121904.*$':'F111214ALH',
'^MORTGAGE EXPRESS\s*200754471.*$|^TOPAZ FIN.*190885105.*$':'F101214ALH',
'^MORTGAGE EXPRESS\s*200754525.*$|^TOPAZ FIN ROSINCA.*134467209.*$':'F141214ALH',
'^MORTGAGE EXPRESS\s*200754534.*$|^TOPAZ FIN.*190885203.*$':'F161214ALH',
'^MORTGAGE EXPRESS\s*200709336.*$|^TOPAZ FIN ROSINCA.*134332909.*$':'F11618ALH',
'^MORTGAGE EXPRESS\s*200709327.*$|^TOPAZ FIN.*190859005.*$':'F31618ALH',
'^PLATFORM FUNDING.*01050230313307.*$':'F61618ALH',
'^MORTGAGE EXPRESS\s*200710767.*$|^TOPAZ FIN.*190859506.*$':'F71618ALH',
'^MORTGAGE EXPRESS\s*200709345.*$|^TOPAZ FIN ROSINCA.*134333009.*$':'F111618ALH',
'^MORTGAGE EXPRESS\s*200878410.*$|^TOPAZ FIN.*190959705.*$':'F121618ALH',
'^MORTGAGE EXPRESS\s*200710776.*$|^TOPAZ FIN ROSINCA.*134338410.*$':'F131618ALH',
'^MORTGAGE EXPRESS\s*200766478.*$|^TOPAZ FIN.*190892600.*$':'F161618ALH',
}

In [31]:
import re

def allocate_codes_for_mortgages(df_bank):
    # Allocate property IDs to Bank payments from Beals
    for index, row in df_bank[(df_bank.Subcategory.str.contains('DIRECTDEBIT|OTH|PAYMENT',case=False,regex=True))&(df_bank.Property.isnull())].iterrows():
        for key in mortgage_map:
            if(re.match(key,row.Memo)):
                df_bank.at[index,'Property']=mortgage_map[key]


In [32]:
allocate_codes_for_mortgages(df6)
allocate_codes_for_mortgages(df3)

In [33]:
# Check there are no mortgages without a property code
df6[(df6.Category=='Mortgage')&(df6.Property.isna())]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category


In [34]:
# Check there are no mortgages without a property code
df3[(df3.Category=='Mortgage')&(df3.Property.isna())]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category
288,2020-09-09,20-53-97 30728691,-340.6,PAYMENT,SIBERITE MORTGAGES ON 08 SEP BCC,NaN,Missed mortage payment?,Mortgage
289,2020-09-09,20-53-97 30728691,-340.6,PAYMENT,SIBERITE MORTGAGES ON 08 SEP BCC,NaN,Missed mortage payment?,Mortgage


#### Allocate Property Codes to Beals Rents

In [35]:
propertyidmap = {
'FLAT 1321 LONDON':'F1321LON',
'FLAT 3321 LONDON':'F3321LON',
'FLAT 4321 LONDON':'F4321LON',
'FLAT 1169 FAWCETT':'F1169FAW',
'FLAT 3169 FAWCETT':'F3169FAW',
'FLAT 1171 FAWCETT':'F1171FAW',
'FLAT 2171 FAWCETT':'F2171FAW',
'FLAT 3163 FRATTON':'F3163FRA',
'FLAT 4163 FRATTON':'F4163FRA',
'SHOP196 KINGSTON': 'SHOP196KIN',
'196A KINGSTON': '196AKIN',
'FLAT 68 ALHAM': 'F68ALH',
'FLAT 78 ALHAM': 'F78ALH',
'FLAT 88 ALHAM': 'F88ALH',
'FLAT 312-14 ALHAM': 'F31214ALH',
'FLAT 412-14 ALHAM': 'F41214ALH',
'FLAT 512-14 ALHAM': 'F51214ALH',
'FLAT 712-14 ALHAM': 'F71214ALH',
'FLAT 1012-14 ALHAM': 'F101214ALH',
'FLAT 1412-14 ALHAM': 'F141214ALH',
'FLAT 1612-14 ALHAM': 'F161214ALH',
'FLAT 616-18 ALHAM': 'F61618ALH',
'FLAT 716-18 ALHAM': 'F71618ALH',
'FLAT 1116-18 ALHAM': 'F111618ALH',
'FLAT 1216-18 ALHAM': 'F121618ALH',
'FLAT 1316-18 ALHAM': 'F131618ALH',
'FLAT 1616-18 ALHAM': 'F161618ALH'
}

In [36]:
def allocate_codes_for_beals_rents(df_bank):
    # Allocate property IDs to Bank payments from Beals
    for index, row in df_bank[df_bank['Memo'].str.contains("BEALS ESTATE AGENT")].iterrows():
        for key in propertyidmap:
            if(key in row.Memo):
                df_bank.at[index,'Property']=propertyidmap[key]
                #df_bank.at[index,'Memo']=propertyidmap[key] + ': ' + df_bank.at[index,'Memo']

In [37]:
allocate_codes_for_beals_rents(df6)
allocate_codes_for_beals_rents(df3)

In [38]:
# Check there are no Beals rents without a property code
df6[(df6['Memo'].str.contains("BEALS ESTATE AGENT"))&(df6.Property.isna())]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category


In [39]:
# Check there are no Beals rents without a property code
df3[(df3['Memo'].str.contains("BEALS ESTATE AGENT"))&(df3.Property.isna())]

,Date,Account,Amount,Subcategory,Memo,Property,Description,Category


### Output Files

In [40]:
file6=backup_file(path+'6045_categorised.xlsx')
file3=backup_file(path+'3072_categorised.xlsx')
df6.to_excel(file6,sheet_name='6045', index=False)
df3.to_excel(file3,sheet_name='3072', index=False)

### Manual Checks
- check categories, particularly Drawings and Intercompany
- run reconciliation notebook out output files
- check for Bank records not reconciled to Beals payment
- check for Beals records with not corresponding Bank payment
- filter bank spreadsheets on property and compare to PropertyApp export
- generate account journals